This notebook contains the basic implementations to replicate the results from the original paper using the provided check points.

In [16]:
# clone the repository (forked from the original one)
!git clone https://github.com/cuevascarlos/CoVR.git

In [3]:
# some adjustements to keep everything consistent
!cp -r CoVR/configs ./
!cp -r CoVR/tools ./
!cp -r CoVR/src ./
!cp -r CoVR/test.py ./
!cp -r CoVR/train.py ./

In [5]:
# install requirements (if using colab)
!python -m pip install -r CoVR/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 25.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multip

In [6]:
# download annotations if needed
!bash CoVR/tools/scripts/download_annotation_cirr.sh

In [8]:
# load the drive (if using colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Compute BLIPS2 embeddings for train, dev and test sets (uncomment those you need to compute)

In [ ]:
# !python tools/embs/save_blip2_embs_imgs.py --image_dir drive/MyDrive/CIRR/images/train --save_dir drive/MyDrive/CIRR/blip2-embs-large/train

In [ ]:
# !python tools/embs/save_blip2_embs_imgs.py --image_dir drive/MyDrive/CIRR/images/test1 --save_dir drive/MyDrive/CIRR/images/test1_blip2

In [3]:
# !python tools/embs/save_blip2_embs_imgs.py --image_dir drive/MyDrive/CIRR/images/dev --save_dir drive/MyDrive/CIRR/blip2-embs-large/dev

Generate blip embeddings for train, dev and test sets (uncomment those you need to compute)

In [ ]:
# !python tools/embs/save_blip_embs_imgs.py --image_dir drive/MyDrive/CIRR/images/train/ --save_dir drive/MyDrive/CIRR/blip-embs-large/train

In [17]:
# !python tools/embs/save_blip_embs_imgs.py --image_dir drive/MyDrive/CIRR/images/test1/ --save_dir drive/MyDrive/CIRR/blip-embs-large/test1

In [ ]:
# !python tools/embs/save_blip_embs_imgs.py --image_dir drive/MyDrive/CIRR/images/dev/ --save_dir drive/MyDrive/CIRR/blip-embs-large/dev

The next cell is used to download the checkpoints to replicate the results from the paper. We reproduced so far the following checkpoints:

- BLIP (1) + CIRR (WebVid-CoVR) (3) (i.e, BLIP pretrained in WebVid-CoVR and fine tuned in CIRR)
- BLIP 2 (2) + WebVid-CoVR+CC-CoIR (2) (i.e, BLIP2 pretrained on WebVid-CoVR+CC-CoIR)
- BLIP 2 (2) + CIRR (WebVid-CoVR+CC-CoIR) (5) (i.e, BLIP2 pretrained on WebVid-CoVR+CC-CoIR and fine tuned in CIRR)

In [22]:
!bash tools/scripts/download_pretrained_models.sh

Select the model to download:
1) BLIP 1 (CoVR)
2) BLIP 2 (CoVR-2)
Press Enter for default (BLIP 2)
Enter your choice (1/2): 2

Select the BLIP 2 model finetuned with dataset:
1) All
2) WebVid-CoVR + CC-CoIR
3) CC-CoIR
4) WebVid-CoVR
5) CIRR (WebVid-CoVR + CC-CoIR)
6) CIRR (CC-CoIR)
7) FashionIQ (WebVid-CoVR + CC-CoIR)
8) FashionIQ (CC-CoIR)
Press Enter for default (All)
Enter your choice (1/2/3/4/5/6/7/8): 5
outputs/cirr/blip2- 100%[===================>]   5.58G  41.2MB/s    in 2m 35s  
Download successful.


In [20]:
# Run this cell to replicate the results for the first option mentioned before
import os

os.environ["HYDRA_FULL_ERROR"] = '1'

!python test.py test=cirr data=cirr model=blip-large model/ckpt=cirr_ft-covr+gt trainer=gpu trainer.devices=1 machine=server machine.batch_size=256 trainer/logger=csv

In [21]:
# Run this cell for the second option mentioned before
import os

os.environ["HYDRA_FULL_ERROR"] = '1'

!python test.py test=cirr data=cirr model=blip2-coco model/ckpt=blip2-l-coco_coir+covr trainer=gpu trainer.devices=1 machine=server machine.batch_size=128 trainer/logger=csv

In [35]:
!cp -r outputs/test drive/MyDrive/CIRR/

In [13]:
# Run this cell to replicate the results for the third option mentioned (BLIP2 + fine tuning in CIRR)
# So far we must change the checkpoint folder names in order to run it properly (check if we can do it directly with the options)
import os

os.environ["HYDRA_FULL_ERROR"] = '1'

!python test.py test=cirr data=cirr model=blip2-coco model/ckpt=blip2-l-coco_coir+covr trainer=gpu trainer.devices=1 machine=server machine.batch_size=128 trainer/logger=csv

2024-12-26 21:38:52.835128: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-26 21:38:52.852099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-26 21:38:52.874190: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-26 21:38:52.880934: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-26 21:38:52.896763: I tensorflow/core/platform/cpu_feature_guar

In [14]:
# save the test outputs in the drive
!cp -r outputs/test/ drive/MyDrive/CIRR/